In [21]:
import pandas as pd
import numpy as np
import os
import s3fs

os.environ["AWS_PROFILE"] = "wri-main"

root = "s3://gfw2-data/climate/LUC_emission_factors/LUC_Emission_factors"
gasses = ["CH4", "CO2", "CO2e", "N2O"]

In [4]:
#################################################################################
# ChatGPT code below to melt the year columns (from like EF_2020, to a year column and EF column)
# Not the nicest be did not feel spending brain energy on this
#################################################################################
def melt_year_values(df):
    # Example: your dataframe is df
    value_cols = [c for c in df.columns if any(x in c for x in ['LD_', 'EF_', 'production_'])]
    
    # Melt the wide columns into long format
    df_long = df.melt(
        id_vars=['aoi_id', 'crop_type', 'gas_type'],
        value_vars=value_cols,
        var_name='variable_year',
        value_name='value'
    )
    
    # Split the variable and year parts
    df_long[['variable', 'year']] = df_long['variable_year'].str.split('_', n=1, expand=True)
    
    # Pivot so each variable (LD, EF, production) becomes its own column
    df_tidy = df_long.pivot_table(
        index=['aoi_id', 'crop_type', 'gas_type', 'year'],
        columns='variable',
        values='value'
    ).reset_index()
    
    # Optional: make sure year is integer
    df_tidy['year'] = df_tidy['year'].astype(int)
    
    # Clean up MultiIndex column names
    df_tidy.columns.name = None

    return df_tidy


In [17]:
# Create an S3 filesystem object
fs = s3fs.S3FileSystem(anon=False)

# List all files under a directory (prefix)
files = fs.ls(f"{root}/deforestation_emission_factors_admin0/")
csv_files = [f for f in files if f.endswith('.csv')]
csv_files

['gfw2-data/climate/LUC_emission_factors/LUC_Emission_factors/deforestation_emission_factors_admin0/emission_factors_CH4_ADM0_master.csv',
 'gfw2-data/climate/LUC_emission_factors/LUC_Emission_factors/deforestation_emission_factors_admin0/emission_factors_CO2_ADM0_master.csv',
 'gfw2-data/climate/LUC_emission_factors/LUC_Emission_factors/deforestation_emission_factors_admin0/emission_factors_CO2e_ADM0_master.csv',
 'gfw2-data/climate/LUC_emission_factors/LUC_Emission_factors/deforestation_emission_factors_admin0/emission_factors_N2O_ADM0_master.csv']

In [5]:
adm0_dfs = []
for gas in gasses:
    uri = f"{root}/deforestation_emission_factors_admin0/emission_factors_{gas}_ADM0_master.csv"
    df = pd.read_csv(uri)
    df["gas_type"] = gas
    adm0_dfs.append(df)
    
adm0_df = pd.concat(adm0_dfs)
adm0_df = adm0_df.rename(columns={"GID_0": "aoi_id"})
adm0_df = melt_year_values(adm0_df)
adm0_df = adm0_df.rename(columns={"EF": "emission_factor"})
adm0_df["aoi_type"] = "admin"
adm0_df

,aoi_id,crop_type,gas_type,year,emission_factor,LD,production,aoi_type
0,AFG,BARL,CH4,2020,0.000260,0.186793,717.372522,admin
1,AFG,BARL,CH4,2021,0.000373,0.145826,390.896728,admin
2,AFG,BARL,CH4,2022,0.000199,0.110480,554.012023,admin
3,AFG,BARL,CH4,2023,0.000126,0.080494,638.620957,admin
4,AFG,BARL,CH4,2024,0.000096,0.052615,546.219402,admin
...,...,...,...,...,...,...,...,...
93075,ZWE,WHEA,N2O,2020,0.001840,206.695135,112353.555832,admin
93076,ZWE,WHEA,N2O,2021,0.001021,182.061850,178265.438089,admin
93077,ZWE,WHEA,N2O,2022,0.001187,156.882132,132162.377118,admin
93078,ZWE,WHEA,N2O,2023,0.000842,133.583707,158593.659530,admin


In [33]:
adm1_dfs = []
for gas in gasses:
    uri = f"{root}/deforestation_emission_factors_adm1/emission_factors_{gas}_ADM1_master.csv"
    df = pd.read_csv(uri)
    df["gas_type"] = gas
    adm1_dfs.append(df)
    
adm1_df = pd.concat(adm1_dfs)
adm1_df["aoi_id"] = adm1_df.GID_1.apply(lambda gadm_id: str(gadm_id).split("_")[0])
adm1_df = adm1_df.drop(columns=["GID_0", "GID_1"])
adm1_df = melt_year_values(adm1_df)
adm1_df["aoi_type"] = "admin"
adm1_df = adm1_df.rename(columns={"EF": "emission_factor"})

# getting 'nan' values for disputed territories with no adm1
adm1_df = adm1_df[adm1_df["aoi_id"] != "nan"]
adm1_df

,aoi_id,crop_type,gas_type,year,EF,LD,production,aoi_type
0,AFG.1,BARL,CH4,2020,0.000000,0.000000,2.071029,admin
1,AFG.1,BARL,CH4,2021,0.000000,0.000000,1.055348,admin
2,AFG.1,BARL,CH4,2022,0.000000,0.000000,1.534942,admin
3,AFG.1,BARL,CH4,2023,0.000000,0.000000,1.783176,admin
4,AFG.1,BARL,CH4,2024,0.000000,0.000000,1.500854,admin
...,...,...,...,...,...,...,...,...
1179435,ZWE.9,WHEA,N2O,2020,0.002050,1.043109,508.772489,admin
1179436,ZWE.9,WHEA,N2O,2021,0.001267,1.022983,807.130416,admin
1179437,ZWE.9,WHEA,N2O,2022,0.001542,0.923840,599.018894,admin
1179438,ZWE.9,WHEA,N2O,2023,0.001162,0.834065,717.952960,admin


In [36]:
adm2_dfs = []
for gas in gasses:
    uri = f"{root}/deforestation_emission_factors_adm2/emission_factors_{gas}_ADM2_master.csv"
    df = pd.read_csv(uri)
    df["gas_type"] = gas
    adm2_dfs.append(df)
    
adm2_df = pd.concat(adm2_dfs)
adm2_df["aoi_id"] = adm2_df.GID_2.apply(lambda gadm_id: str(gadm_id).split("_")[0])
adm2_df = adm2_df.drop(columns=["GID_0", "GID_1", "GID_2"])
adm2_df = melt_year_values(adm2_df)
adm2_df["aoi_type"] = "admin"
adm2_df = adm2_df.rename(columns={"EF": "emission_factor"})
# getting 'nan' values for disputed territories with no adm1
adm2_df = adm2_df[adm2_df["aoi_id"] != "nan"]
adm2_df

,aoi_id,crop_type,gas_type,year,emission_factor,LD,production,aoi_type
0,AFG.1.1,POTA,CH4,2020,0.000000,0.000000,0.025861,admin
1,AFG.1.1,POTA,CH4,2021,0.000000,0.000000,0.026585,admin
2,AFG.1.1,POTA,CH4,2022,0.000000,0.000000,0.026877,admin
3,AFG.1.1,POTA,CH4,2023,0.000000,0.000000,0.026877,admin
4,AFG.1.1,POTA,CH4,2024,0.000000,0.000000,0.026803,admin
...,...,...,...,...,...,...,...,...
11829835,ZWE.9.9,OPUL,N2O,2020,0.001047,0.000102,0.097291,admin
11829836,ZWE.9.9,OPUL,N2O,2021,0.000948,0.000092,0.097291,admin
11829837,ZWE.9.9,OPUL,N2O,2022,0.000848,0.000082,0.097291,admin
11829838,ZWE.9.9,OPUL,N2O,2023,0.000748,0.000073,0.097291,admin


In [72]:
os.environ["AWS_PROFILE"] = "zeno"

final_df = pd.concat([adm0_df, adm1_df, adm2_df])
final_df = final_df.rename(columns={"emission_factor": "emissions_factor", "LD": "emissions_tonnes", "production": "production_tonnes"})

code_to_crop = {
    "ACOF": "Coffee",
    "AOIL": "Oil palm",
    "AOLS": "Olives",
    "APPL": "Apples",
    "AVOC": "Avocados",
    "BANA": "Bananas",
    "BARC": "Barley",
    "BEAN": "Beans",
    "CASS": "Cassava",
    "CERE": "Other cereals",
    "CHIC": "Chickpeas",
    "COCN": "Coconuts",
    "COCO": "Cocoa",
    "COTT": "Cotton",
    "COWP": "Cowpeas",
    "GCOF": "Groundnuts (peanuts)",
    "MAIZ": "Maize (corn)",
    "MILL": "Millet",
    "OATS": "Oats",
    "ONIO": "Onions",
    "OPUL": "Other pulses",
    "OVEG": "Other vegetables",
    "PEAS": "Peas",
    "POTA": "Potatoes",
    "RAPE": "Rapeseed (canola)",
    "RICE": "Rice",
    "RYEM": "Rye",
    "SESM": "Sesame",
    "SORG": "Sorghum",
    "SOYB": "Soybeans",
    "SUGB": "Sugar beet",
    "SUGC": "Sugarcane",
    "SUNF": "Sunflower",
    "SWPO": "Sweet potatoes",
    "TOBA": "Tobacco",
    "TOMT": "Tomatoes",
    "TROF": "Other tropical fruits",
    "WHEA": "Wheat",
    "YAMS": "Yams",
    "BARL": "Barley",
    "CNUT": "Coconut",
    "GROU": "Groundnut (with shell)",
    "LENT": "Lentils",
    "OCER": "Other cereals",
    "OFIB": "Other fibre crops",
    "OILP": "Oil palm",
    "OOIL": "Other oil crops",
    "ORTS": "Other roots & tubers",
    "PIGE": "Pigeon pea",
    "PLNT": "Plantain",
    "PMIL": "Pearl millet",
    "RCOF": "Coffee (Robusta)",
    "REST": "Rest of crops",
    "SESA": "Sesame",
    "SMIL": "Small millet",
    "TEAS": "Tea",
    "TEMF": "Temperate fruits",
    "VEGE": "Vegetables",
}

# Add a column with the full name
final_df["crop_type"] = final_df["crop_type"].map(code_to_crop)

final_df.to_parquet("s3://lcl-analytics/zonal-statistics/admin-deforestation-emissions-by-crop.parquet", index=False)
final_df

,aoi_id,crop_type,gas_type,year,emissions_factor,emissions_tonnes,production_tonnes,aoi_type
0,AFG,Barley,CH4,2020,0.000260,0.186793,717.372522,admin
1,AFG,Barley,CH4,2021,0.000373,0.145826,390.896728,admin
2,AFG,Barley,CH4,2022,0.000199,0.110480,554.012023,admin
3,AFG,Barley,CH4,2023,0.000126,0.080494,638.620957,admin
4,AFG,Barley,CH4,2024,0.000096,0.052615,546.219402,admin
...,...,...,...,...,...,...,...,...
11829835,ZWE.9.9,Other pulses,N2O,2020,0.001047,0.000102,0.097291,admin
11829836,ZWE.9.9,Other pulses,N2O,2021,0.000948,0.000092,0.097291,admin
11829837,ZWE.9.9,Other pulses,N2O,2022,0.000848,0.000082,0.097291,admin
11829838,ZWE.9.9,Other pulses,N2O,2023,0.000748,0.000073,0.097291,admin


In [ ]:
final_df[((final_df["aoi_id"] == "BRA.12.3")|(final_df["aoi_id"] == "IDN.24.9")) & ((final_df["crop_type"] == "Bananas") | (final_df["crop_type"] == "Yams"))].to_dict(orient="list")